In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Selenium için Chrome sürücüsünü başlatma
chrome_options = Options()
chrome_options.add_argument("--headless")  # Tarayıcıyı başlık olmadan çalıştırma
chrome_options.add_argument("--disable-gpu")  # GPU'yu devre dışı bırakma (başlık modunda gerekli olabilir)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

comments = []
scroll_pause_time = 3  # Sayfanın yüklenmesi için bekleme süresi

# 1'den 350'ye kadar sayfaları dolaşma
for page in range(1, 351):
    url = f"https://www.sikayetvar.com/operator?page={page}"
    driver.get(url)

    # Sayfanın tam olarak yüklenmesini beklemek için uyku (sleep) süresi ekleyin
    time.sleep(scroll_pause_time)  # Bu süreyi gerekirse artırabilirsiniz

    # Yorumları toplama
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Sayfanın sonuna kaydır
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Yorumları bulma
        complaints = driver.find_elements(By.CLASS_NAME, "complaint-description")
        for complaint in complaints:
            comments.append(complaint.text.strip())

        # Bekleme
        time.sleep(scroll_pause_time)

        # Yeni sayfa yüksekliğini hesapla ve karşılaştır
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Tarayıcıyı kapatma
driver.quit()

# Yorumları benzersiz hale getirme (tekrarlanan yorumları kaldırma)
comments = list(set(comments))

# Veri setini oluşturma
data = pd.DataFrame(comments, columns=["comment"])

# Yorumları etiketleme
def label_sentiment(comment):
    positive_words = ["iyi", "mükemmel", "harika", "güzel", "memnun"]
    negative_words = ["kötü", "berbat", "rezalet", "korkunç", "memnun değilim"]

    comment = comment.lower()
    if any(word in comment for word in positive_words):
        return 'olumlu'
    elif any(word in comment for word in negative_words):
        return 'olumsuz'
    else:
        return 'nötr'

data['sentiment'] = data['comment'].apply(label_sentiment)

# Veri çerçevesinin satır sayısını yazdırma
print(f"Toplam yorum sayısı: {data.shape[0]}")

# CSV dosyasına yazma
data.to_csv("/content/drive/MyDrive/oflazer/Mert/sikayetvar_comments_labeled.csv", index=False, encoding='utf-8')

print(data.head())